In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/aes2-0-self-train-model/checkpoint-2000/checkpoint-2000/model-2000.pth
/kaggle/input/aes2-0-self-train-model/checkpoint-1700/checkpoint-1700/model-1700.pth
/kaggle/input/learning-agency-lab-automated-essay-scoring-2/sample_submission.csv
/kaggle/input/learning-agency-lab-automated-essay-scoring-2/train.csv
/kaggle/input/learning-agency-lab-automated-essay-scoring-2/test.csv
/kaggle/input/deberta-v3-large/spm.model
/kaggle/input/deberta-v3-large/config.json
/kaggle/input/deberta-v3-large/pytorch_model.generator.bin
/kaggle/input/deberta-v3-large/README.md
/kaggle/input/deberta-v3-large/tf_model.h5
/kaggle/input/deberta-v3-large/tokenizer_config.json
/kaggle/input/deberta-v3-large/gitattributes
/kaggle/input/deberta-v3-large/pytorch_model.bin
/kaggle/input/deberta-v3-large/generator_config.json


In [1]:
from transformers.modeling_outputs import SequenceClassifierOutputWithPast
from transformers import (
    AutoModelForSequenceClassification,
    AutoModel,
    AutoConfig,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    AdamW,
    get_polynomial_decay_schedule_with_warmup
)
from tqdm import tqdm
import torch.nn as nn
import torch

/ai/anaconda3/envs/bst/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-05-05 15:38:45.012257: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-05 15:38:45.224121: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-05 15:38:46.149988: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: 

In [2]:
def get_last_hidden_state(backbone_outputs):
    last_hidden_state = backbone_outputs[0]
    return last_hidden_state


def get_all_hidden_states(backbone_outputs):
    all_hidden_states = torch.stack(backbone_outputs[1])
    return all_hidden_states


def get_input_ids(inputs):
    return inputs["input_ids"]


def get_attention_mask(inputs):
    return inputs["attention_mask"]

In [4]:
class MeanPooling(nn.Module):
    def __init__(self):
        super(MeanPooling, self).__init__()
        self.output_dim = AutoConfig.from_pretrained("/kaggle/input/deberta-v3-large").hidden_size

    def forward(self, inputs, backbone_outputs):
        attention_mask = get_attention_mask(inputs)
        last_hidden_state = get_last_hidden_state(backbone_outputs)

        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        return mean_embeddings

class CustomModel(nn.Module):
    def __init__(self, tokenizer):
        super().__init__()
        self.backbone = AutoModel.from_pretrained("/kaggle/input/deberta-v3-large")
        self.backbone.resize_token_embeddings(len(tokenizer))
        self.pool = MeanPooling()
        self.fc = nn.Linear(self.pool.output_dim, 1)

    def forward(self, inputs):
        outputs = self.backbone(**inputs)
        feature = self.pool(inputs, outputs)
        output = self.fc(feature)

        return SequenceClassifierOutputWithPast(
            loss=None,
            logits=output,
            past_key_values=None,
            hidden_states=None,
            attentions=None
        )
    
tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/deberta-v3-large")
tokenizer.add_special_tokens(
    {"additional_special_tokens": ["[PARAGRAPH]"]}
)
print(len(tokenizer))
model = CustomModel(tokenizer=tokenizer)
print(model)

/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


128002


/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


CustomModel(
  (backbone): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128002, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-23): 24 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (key_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (value_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-07, elementwise_affine=True)
              (drop

In [5]:
model.load_state_dict(torch.load("/kaggle/input/aes2-0-self-train-model/checkpoint-2000/checkpoint-2000/model-2000.pth"))
model = model.to("cuda")

In [6]:
df = pd.read_csv("/kaggle/input/learning-agency-lab-automated-essay-scoring-2/test.csv")
df["full_text"] = df["full_text"].str.replace(r'\n\n', "[PARAGRAPH]", regex=True)

In [7]:
model.eval()
predictions = []

for idx, row in tqdm(df.iterrows(), total=len(df)):
    text = row["full_text"]
    encoded_input = tokenizer(text, max_length=1800, truncation=True, padding="max_length", return_tensors="pt")
    encoded_input = encoded_input.to("cuda")
    prediction = round(np.clip(model(encoded_input).logits.cpu().detach().numpy().tolist()[0][0], 1, 6))
    print(prediction)
    predictions.append(prediction)

 33%|███▎      | 1/3 [00:02<00:04,  2.09s/it]

2


 67%|██████▋   | 2/3 [00:03<00:01,  1.42s/it]

3


100%|██████████| 3/3 [00:04<00:00,  1.34s/it]

5


In [8]:
submission = pd.read_csv("/kaggle/input/learning-agency-lab-automated-essay-scoring-2/sample_submission.csv")
submission

,essay_id,score
0,000d118,3
1,000fe60,3
2,001ab80,4


In [9]:
submission["score"] = predictions
submission.to_csv("submission.csv", index=False)